<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
# youtube video mAP implementation
import torch
from collections import Counter

def mean_avg_precision(pred_bbs, gt_bbs, n_cls, iou_thresh=.5):
    """Source: https://www.youtube.com/watch?v=FppOzcDvaDI"""
    # pred_bbs[0]: [im_idx, pred_lbl, conf, *pred_bb]
    # gt_bbs[0]:   [im_idx, gt_lbl,   *pred_bb]
    avg_precisions = []
    eps = 1e-6 # for numerical stability
    
    for c in range(n_cls):
        # store pred_bbs and gt_bbs for current class
        preds = []
        gts = []
        
        # grab preds and gts for current class
        for pred_bb in pred_bbs:
            if pred_bb[1] == c: preds.append(pred_bb)
        for gt_bb in gt_bbs:
            if gt_bb[1] == c: gts.append(gt_bb)
                
        # sort preds by conf desc
        preds.sort(key=lambda x: x[2], reverse=True)
        
        # make dict of im_idx:zeros(n_objs)
        # im0 has 1 bb and im1 has 3 bbs
        # amount_bbs = {0:1, 1:3}
        # amount_bbs = {0:tensor([0]), 1:tensor([0,0,0])}
        amount_bbs = Counter([gt[0] for gt in gts])
        for key,val in amount_bbs.items():
            amount_bbs[key] = torch.zeros(val)
        
        # init tp and fp
        tp = torch.zeros((len(preds)))
        fp = torch.zeros((len(preds)))
        total_true_bbs = len(gts)
        
        for pred_idx, pred in enumerate(preds):
            gt_im = [bb for bb in gts if bb[0] == pred[0]]
            n_gts = len(gt_im)
            best_iou = 0       
            
            for idx, gt in enumerate(gt_im):
                iou = get_iou(tensor(pred[3:]), tensor(gt[3:]))
                if iou > best_iou: best_iou,best_gt_idx = iou,idx
            
            # update idx of gtbb to indicate it's been used
            if best_iou > iou_thresh:
                if amount_bbs[pred[0]][best_gt_idx]==0:
                    tp[pred_idx] = 1
                    amount_bbs[pred[0]][best_gt_idx] = 1
                else: fp[pred_idx] = 1
            else: fp[pred_idx] = 1
        
        # [1,1,0,1,0] → [1,2,2,3,3]
        tp_csum = torch.cumsum(tp, dim=0)
        fp_csum = torch.cumsum(fp, dim=0)
        recalls = tp_csum / (total_true_bbs + eps)
        precisions = torch.divide(tp_csum, (tp_csum + fp_csum + eps))
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        avg_precision.append(torch.trapz(precisions, recalls))
        
    return sum(avg_precisions)/ len(avg_precisions)